In [14]:
import sys
import os
import dill
sys.path.append('/Users/jshleap/Playground/Insight/HiddenKeywords')
from scripts.gimmewords import *
import pandas as pd
mock_landing_page = "https://blog.paperspace.com/tag/machine-learning/" 
mock_GKP_result = "/Users/jshleap/Playground/Insight/HiddenKeywords/data/Paperspace_KW_Stats_2019-09-23_at_14_37_49.csv"
max_search = 10 
max_results = 5 
model = 'lda'

In [15]:
df_opt = dict(skiprows=[0, 1], encoding=detect_encoding(mock_GKP_result), sep='\t')
gkp = pd.read_csv(mock_GKP_result, **df_opt)
gkp

,Keyword,Currency,Avg. monthly searches,Min search volume,Max search volume,Competition,Competition (indexed value),Top of page bid (low range),Top of page bid (high range),Ad impression share,...,Searches: Nov 2018,Searches: Dec 2018,Searches: Jan 2019,Searches: Feb 2019,Searches: Mar 2019,Searches: Apr 2019,Searches: May 2019,Searches: Jun 2019,Searches: Jul 2019,Searches: Aug 2019
0,papersapce,CAD,NaN,100,"1,000",Low,11.0,3.32,15.12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,paperspace,CAD,NaN,"10,000","100,000",Low,12.0,3.97,12.73,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,paperspace ai,CAD,NaN,10,100,Low,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,paperspace deep learning,CAD,NaN,10,100,Low,18.0,2.53,6.63,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,paperspace machine,CAD,NaN,10,100,Low,5.0,7.44,20.65,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,paperspace virtual machine,CAD,NaN,10,100,Low,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,paperspace student,CAD,NaN,10,100,Low,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,paperspace machine learning,CAD,NaN,10,100,Medium,33.0,3.32,19.92,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,free cloud deep learning,CAD,NaN,10,100,Medium,39.0,2.67,12.93,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,paperspace free,CAD,NaN,10,100,Low,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# I preran the crawler for speedsake
path = os.path.abspath(os.path.join(os.getcwd(), os.path.pardir))
with open(os.path.join(path, 'pages.dmp')) as p, open(os.path.join(path, 'landing.dmp')) as l:
    text = [line for line in p]
    land = ' '.join([line for line in l])
land[:10]

'august dee'

In [21]:
# I pre-ran the class 
with open(os.path.join(path, 'iw.pkcl'), 'rb') as p:
    iw = dill.load(p)

In [23]:
print('Corpus Keywords', iw.pre_keywords[0][:10])
print('Landing page Keywords', iw.landing_kw[:10])
print('GKP keywords', iw.gkp.Keyword.head(10))


Corpus Keywords [('imaging', 0.21977344255357842), ('modeling', 0.19233786074451106), ('trained', 0.1598793969874851), ('train', 0.1598793969874851), ('trains', 0.1598793969874851), ('learn', 0.1595379713518971), ('like', 0.1592958608388017), ('learns generalize', 0.15575554036108102), ('networks', 0.15399402972809328), ('network', 0.15399402972809328)]
Landing page Keywords [('learns', 0.18931508923314405), ('image', 0.1883491301552187), ('imaging', 0.1883491301552187), ('gradients', 0.16590636782313423), ('learn train', 0.16365073881279016), ('functional', 0.16166409086181452), ('functioning', 0.16166409086181452), ('models', 0.15016127448429095), ('modeling', 0.15016127448429095), ('likely', 0.1500623402733657)]
GKP keywords 0                     papersapce
1                     paperspace
2                  paperspace ai
3       paperspace deep learning
4             paperspace machine
5     paperspace virtual machine
6             paperspace student
7    paperspace machine learnin

In [28]:
bag_of_keywords = set(x[0] for y in iw.pre_keywords for x in y)
keywords_not_gkp = bag_of_keywords.difference(x[0] for x in iw.landing_kw)
print(len(keywords_not_gkp))
print(list(keywords_not_gkp)[:10])

1114
['minute', 'embedding', 'aim', 'container', 'methods optimizations', 'folder', 'growing base', 'easy experiment', 'human animal', 'servers']


In [42]:
iw = IdentifyWords(text, mock_GKP_result, land, max_df=0.9, min_df=0.01,
                 max_features=100, n_keywords=10, model='word2vec')


/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'common', 'countrynames', 'html', 'stopwords', 'useful', 'usual', 'vis', 'viser', 'visest', 'words'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Top 10 words in vocabulary
['predictions', 'maps', 'important', 'required', 'space', 'architectures', 'non', 'partial', 'credits', 'blocks']


In [43]:
w2v = iw.word2vec()

In [45]:
w2v.most_similar(positive='methods optimizations ')


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'methods optimizations ' not in vocabulary"

In [37]:
w2v.build_vocab(new, update=True)

In [40]:
w2v.train(new, total_examples=w2v.corpus_count, epochs=w2v.epochs)

ValueError: You must specify an explict epochs count. The usual value is epochs=model.epochs.

In [41]:
text

['april tutorial detecting localizing pneumonia chest ray scans pytorch years seen powerful models built distinguish objects models keep terms performance latency day day wondered exactly models pick images train practically flawless predictions undoubtedly features images feed models look predictions seek explore article long ago researchers stanford university released paper https arxiv org abs using deep learning push edge pneumonia diagnosis work fascinated tried pytorch going show implemented work using dataset kaggle link paper class activation maps http cnnlocalization csail mit edu zhou tutorial neural machine translation tensorflow fan translate translation service wonder programs able spot translations language par henry ansah fordjour min read series gradient descent python part generic python implementation gradient descent nn optimization hello series tutorials implementing generic gradient descent gd algorithm python optimizing parameters artificial neural network ann bac

In [16]:
iw = IdentifyWords(text, mock_GKP_result, land, max_df=0.9, min_df=0.01, max_features=100, n_keywords=10, 
                   model='lda')
dictionary, corpus, lda = iw.lda()

Top 10 words in vocabulary
['technique', 'still', 'module', 'big', 'may', 'git', 'repo', 'tensor', 'job', 'torch']


CalledProcessError: Command '/Users/jshleap/Playground/Insight/HiddenKeywords/scripts import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/zt/p4bj1_l97jl20b3jss0tyz300000gn/T/9525f_corpus.txt --output /var/folders/zt/p4bj1_l97jl20b3jss0tyz300000gn/T/9525f_corpus.mallet' returned non-zero exit status 126.

In [ ]:
# select and article at random from train_df
random_article_index = np.random.randint(len(text))
bow = dictionary.doc2bow(text[random_article_index])
# get the topic contributions for the document chosen at random above
doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=bow)])
# print the top 5 contributing topics and their words
for i in doc_distribution.argsort()[-5:][::-1]:
    print(i, lda.show_topic(topicid=i, topn=10), "\n")
new_bow = dictionary.doc2bow(land)
new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
# print the top 8 contributing topics and their words
for i in new_doc_distribution.argsort()[-5:][::-1]:
    print(i, lda.show_topic(topicid=i, topn=10), "\n")
# we need to use nested list comprehension here
# this may take 1-2 minutes...
doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
doc_topic_dist.shape

In [ ]:
def jensen_shannon(query, matrix):
    """
    https://www.kaggle.com/ktattan/lda-and-document-similarity
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix
    m = 0.5*(p + q)
    return np.sqrt(0.5*(entropy(p,m) + entropy(q,m)))


k=10
query, matrix = new_doc_distribution, doc_topic_dist
sims = jensen_shannon(query,matrix) # list of jensen shannon distances
k_top sims.argsort()[:k] 
